# Getting the weights for the regressor

In [3]:
import pandas as pd
import numpy as np
import json
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### importing data

In [4]:
# Opening JSON file
with open('apartments.json') as json_file:
    apartments = json.load(json_file)
 
    # Print the type of data variable
    print("Type:", type(apartments))

Type: <class 'list'>


### making the DataFrame

In [5]:
apts = []
for apartment in apartments:
    apt = {}
    apt["lat"] = apartment["lat"]
    apt["long"] = apartment["long"]
    apt["date"] = apartment["date"]
    apt["sq_mt"] = apartment["sq_mt"]
    apt["rooms"] = apartment["rooms"]
    apt["price"] = apartment["price"]
    apts.append(apt)

In [6]:
apt_df =pd.DataFrame(apts)

In [7]:
apt_df

,lat,long,date,sq_mt,rooms,price
0,44.770200,20.419700,29.08.2022.,134.00,4.5,339000
1,44.798600,20.472300,29.08.2022.,60.00,2.0,169000
2,44.793400,20.492800,29.08.2022.,110.00,3.5,239000
3,44.843383,20.484003,29.08.2022.,49.00,2.5,67800
4,44.755283,20.453176,29.08.2022.,40.00,1.5,73000
...,...,...,...,...,...,...
469,44.776801,20.532719,01.08.2022.,51.82,3.0,101049
470,44.805053,20.474391,01.08.2022.,66.00,3.0,208000
471,44.792964,20.492485,31.07.2022.,42.00,1.0,130000
472,44.795135,20.437267,31.07.2022.,110.00,5.0,140000


In [8]:
# plt.plot(apt_df.long, apt_df.lat, ".")

### Preparing data for ML

In [9]:
X = apt_df.drop(["price", "date"], axis=1).to_numpy()
y = apt_df.price.to_numpy()

#### Linear regressor

In [10]:
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)
linreg = LinearRegression().fit(X_train, y_train)

print('linear model coeff (w): {}'
     .format(linreg.coef_))
print('linear model intercept (b): {:.3f}'
     .format(linreg.intercept_))
print('R-squared score (training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))

linear model coeff (w): [601494.25997073 138160.25398513   5720.21981834 -77965.5288013 ]
linear model intercept (b): -29791316.084
R-squared score (training): 0.737
R-squared score (test): 0.472


#### Linear regressor with ridge

In [18]:
from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)


linridge = Ridge(alpha=100.0).fit(X_train, y_train)

print('Crime dataset')
print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

Crime dataset
ridge regression linear model intercept: -167166.63382314288
ridge regression linear model coeff:
[  2991.04624665  -1239.03267761   5225.87498814 -49602.69764803]
R-squared score (training): 0.724
R-squared score (test): 0.495
Number of non-zero features: 4


In [17]:
print('Ridge regression: effect of alpha regularization parameter\n')
for this_alpha in [0, 1, 10, 20, 50, 100, 1000]:
    linridge = Ridge(alpha = this_alpha).fit(X_train, y_train)
    r2_train = linridge.score(X_train, y_train)
    r2_test = linridge.score(X_test, y_test)
    num_coeff_bigger = np.sum(abs(linridge.coef_) > 1.0)
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
r-squared training: {:.2f}, r-squared test: {:.2f}\n'
         .format(this_alpha, num_coeff_bigger, r2_train, r2_test))
linridge.coef_

Ridge regression: effect of alpha regularization parameter

Alpha = 0.00
num abs(coeff) > 1.0: 4, r-squared training: 0.74, r-squared test: 0.47

Alpha = 1.00
num abs(coeff) > 1.0: 4, r-squared training: 0.73, r-squared test: 0.48

Alpha = 10.00
num abs(coeff) > 1.0: 4, r-squared training: 0.73, r-squared test: 0.48

Alpha = 20.00
num abs(coeff) > 1.0: 4, r-squared training: 0.73, r-squared test: 0.48

Alpha = 50.00
num abs(coeff) > 1.0: 4, r-squared training: 0.73, r-squared test: 0.49

Alpha = 100.00
num abs(coeff) > 1.0: 4, r-squared training: 0.72, r-squared test: 0.49

Alpha = 1000.00
num abs(coeff) > 1.0: 4, r-squared training: 0.70, r-squared test: 0.48



array([   306.83376855,   -139.57289027,   4550.29191328, -11511.21968933])

In [19]:
weights = {}
weights["intercept"] = linridge.intercept_
weights["coefficients"] = list(linridge.coef_)
with open('weights.json', 'w') as file:
    json.dump(weights, file)

#### Linear regressor with ridge and data scaling

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   random_state = 0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

linridge = Ridge(alpha=20.0).fit(X_train_scaled, y_train)

print('Crime dataset')
print('ridge regression linear model intercept: {}'
     .format(linridge.intercept_))
print('ridge regression linear model coeff:\n{}'
     .format(linridge.coef_))
print('R-squared score (training): {:.3f}'
     .format(linridge.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'
     .format(linridge.score(X_test_scaled, y_test)))
print('Number of non-zero features: {}'
     .format(np.sum(linridge.coef_ != 0)))

Crime dataset
ridge regression linear model intercept: 63749.612928978866
ridge regression linear model coeff:
[ 29926.31640935  -9006.59321678 254823.17930099 218096.23811708]
R-squared score (training): 0.293
R-squared score (test): 0.344
Number of non-zero features: 4


#### finding the best alpha parameter

In [10]:
print('Ridge regression: effect of alpha regularization parameter\n')
for this_alpha in [0, 1, 10, 20, 50, 100, 1000]:
    linridge = Ridge(alpha = this_alpha).fit(X_train_scaled, y_train)
    r2_train = linridge.score(X_train_scaled, y_train)
    r2_test = linridge.score(X_test_scaled, y_test)
    num_coeff_bigger = np.sum(abs(linridge.coef_) > 1.0)
    print('Alpha = {:.2f}\nnum abs(coeff) > 1.0: {}, \
r-squared training: {:.2f}, r-squared test: {:.2f}\n'
         .format(this_alpha, num_coeff_bigger, r2_train, r2_test))
linridge.coef_

Ridge regression: effect of alpha regularization parameter

Alpha = 0.00
num abs(coeff) > 1.0: 4, r-squared training: 0.74, r-squared test: 0.47

Alpha = 1.00
num abs(coeff) > 1.0: 4, r-squared training: 0.66, r-squared test: 0.56

Alpha = 10.00
num abs(coeff) > 1.0: 4, r-squared training: 0.38, r-squared test: 0.41

Alpha = 20.00
num abs(coeff) > 1.0: 4, r-squared training: 0.29, r-squared test: 0.34

Alpha = 50.00
num abs(coeff) > 1.0: 4, r-squared training: 0.19, r-squared test: 0.23

Alpha = 100.00
num abs(coeff) > 1.0: 4, r-squared training: 0.12, r-squared test: 0.14

Alpha = 1000.00
num abs(coeff) > 1.0: 4, r-squared training: 0.01, r-squared test: 0.00



array([  689.58641122,  -202.49696275,  7421.00072051, 10360.88115458])

#### Using the best alpha parameter

In [11]:
linridge = Ridge(alpha = 1).fit(X_train_scaled, y_train)
r2_train = linridge.score(X_train_scaled, y_train)
r2_test = linridge.score(X_test_scaled, y_test)
predictions = linridge.predict(X_test_scaled)

In [12]:
new_df = pd.DataFrame()
new_df["predictions"] = predictions
new_df["y_test"] = y_test
new_df.head(50)

In [15]:
X_train_scaled
linridge.coef_

array([ 189080.88062186,   11720.96837901, 1605422.31442403,
         -9944.09381746])

In [16]:
linridge.intercept_

-190078.00275754

In [17]:
X_test_scaled[0]

array([0.68089477, 0.53179326, 0.2262931 , 0.66666667])

In [18]:
y_test[0]

249000

In [19]:
sum1 = []
for i in range(4):

    print(X_test_scaled[0][i])
    print(linridge.coef_[i])
    print("==================")
    mul_res = X_test_scaled[0][i]*linridge.coef_[i]
    print(mul_res)
    print("******************")
    sum1.append(mul_res)
print(sum1)

0.6808947700063044
189080.88062185762
128744.18272360924
******************
0.5317932637487885
11720.968379006605
6233.13202856827
******************
0.22629310344827586
1605422.3144240326
363295.99787612807
******************
0.6666666666666665
-9944.093817455787
-6629.395878303856
******************
[128744.18272360924, 6233.13202856827, 363295.99787612807, -6629.395878303856]


In [21]:
sum(sum1)+linridge.intercept_

301565.91399246175

### Exporting the coeffs and intercept

In [22]:
weights = {}
weights["intercept"] = linridge.intercept_
weights["coefficients"] = list(linridge.coef_)
with open('weights.json', 'w') as file:
    json.dump(weights, file)

#### Neural netwrk

In [30]:
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt



X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

for thisactivation in ['tanh', 'relu']:
    for thisalpha in [0.0001, 1.0, 100]:
        mlpreg = MLPRegressor(hidden_layer_sizes = [100,100],
                             activation = thisactivation,
                             alpha = thisalpha,
                             solver = 'lbfgs').fit(X_train, y_train)
        score = mlpreg.score(X_test, y_test)
        print(str(score) + " where, activation is " + thisactivation + " and alpha is " + str(thisalpha))
        

-0.021154840100237715 where, activation is tanh and alpha is 0.0001


C:\Users\savko\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.2496935224309269 where, activation is tanh and alpha is 1.0


C:\Users\savko\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.22815152657639015 where, activation is tanh and alpha is 100
0.6722270258035464 where, activation is relu and alpha is 0.0001


C:\Users\savko\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.6743898154587042 where, activation is relu and alpha is 1.0
0.685166992093396 where, activation is relu and alpha is 100


In [35]:
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt



X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)



mlpreg = MLPRegressor(hidden_layer_sizes = [100,100],
                     activation = "relu",
                     alpha = 300,
                     solver = 'lbfgs').fit(X_train, y_train)
score = mlpreg.score(X_test, y_test)
print(str(score))

predictions = mlpreg.predict(X_test)
new_df = pd.DataFrame()
new_df["predictions"] = predictions
new_df["y_test"] = y_test
new_df.head(50)

0.6813593339864725


C:\Users\savko\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,predictions,y_test
0,297510.686440,249000
1,253247.439540,175000
2,124579.846680,250000
3,63238.056995,88000
4,209867.784020,305000
5,79751.793013,45000
6,80992.607859,125000
7,300377.500433,350104
8,142626.486431,149000
9,100572.217299,120000
